In [22]:
import pandas as pd
import numpy as np
import os

#get current path
currentPath = os.getcwd()
print(currentPath)

#change path
os.chdir(currentPath)

C:\Users\Lenovo\Desktop\P-Sat\20-1 방세\완성본


In [23]:
dat = pd.read_csv('train.csv')

dat_x = dat.drop(['click', 'event_datetime'], axis = 1)
dat_y = dat.loc[:, ['click']]

In [24]:
test = pd.read_csv('Test_data.csv')

In [25]:
test['event_datetime']=pd.to_datetime(test['event_datetime'], errors='coerce')
test['hour']=test['event_datetime'].dt.hour
test['minute']=test['event_datetime'].dt.minute
test['weekday']= test['event_datetime'].dt.day_name()

import math

mins_in_day = 24 * 60
test['time'] = test['hour'] * 60 + test['minute']
test['sin_time'] = test['time'].apply(lambda x: math.sin(x* math.pi*2 / mins_in_day))
test['cos_time'] = test['time'].apply(lambda x: math.cos(x * math.pi * 2 / mins_in_day))
del test['time']

In [26]:
bid_id = test['bid_id']
test = test.drop(columns = ['bid_id', 'event_datetime', 'device_country', 'hour', 'minute'])

In [27]:
from category_encoders.cat_boost import CatBoostEncoder

def catboost_encoder(train_x, train_y, test_x):
  cate_var = list(train_x.select_dtypes(include=['object', 'int64']))
  encoder = CatBoostEncoder()
  cbe_train_x = encoder.fit_transform(train_x[cate_var], train_y)
  cbe_test_x = encoder.transform(test_x[cate_var])
  cbe_train_result = np.concatenate((cbe_train_x, train_x.loc[:,['sin_time', 'cos_time']]), axis=1)
  cbe_test_result = np.concatenate((cbe_test_x, test_x.loc[:,['sin_time', 'cos_time']]), axis=1)
  cbe_final_train = pd.DataFrame(cbe_train_result, columns= train_x.columns)
  cbe_final_test = pd.DataFrame(cbe_test_result, columns= test_x.columns)
  return(cbe_final_train, cbe_final_test)

In [28]:
train_cbe_x, test_x = catboost_encoder(dat_x, dat_y, test_x = test)

In [29]:
import pickle
import joblib

## 1. Logistic Regression

In [30]:
file_name1 = 'logistic.pkl'
logistic = joblib.load(file_name1)

In [31]:
logistic_pred_y = logistic.predict_proba(test_x)
logistic_pred_y = pd.DataFrame(logistic_pred_y)

## 2. Light GBM

In [32]:
file_name2 = 'lgbm.pkl'
lgbm = joblib.load(file_name2)

In [33]:
lgbm_pred_y = lgbm.predict_proba(test_x)
lgbm_pred_y = pd.DataFrame(lgbm_pred_y)

## 3. Catboost

In [34]:
from catboost import CatBoostClassifier, Pool

In [35]:
catboost_clf = CatBoostClassifier()
catboost_clf.load_model("catboost_clf")

In [36]:
cat_pred_y = catboost_clf.predict_proba(test)
cat_pred_y = pd.DataFrame(cat_pred_y)

In [37]:
# 삭제하고 RAM 광명찾자

del(dat)
del(dat_x)
del(dat_y)
del(train_cbe_x)
del(test)
del(test_x)

## 4. Ensemble

In [38]:
logistic_prob = logistic_pred_y.iloc[:,1].to_list()
lgbm_prob = lgbm_pred_y.iloc[:,1].to_list()
cat_prob = cat_pred_y.iloc[:,1].to_list()

In [39]:
ensemble_pred_y = (
np.asarray([prob * 0.59 for prob in lgbm_prob]) + 
np.asarray([prob*0.4 for prob in cat_prob]) + 
np.asarray([prob*0.01 for prob in logistic_prob])
)

# 넘파이로 빠른 계산

In [40]:
submit = pd.DataFrame()
submit['bid_id'] = np.array(bid_id)
submit['click'] = ensemble_pred_y

In [41]:
submit.head()

bid_id     click
0  Sh0wWllIpg  0.051301
1  gKh5ZZcIAu  0.053732
2  rwnUHAWIck  0.051476
3  0Eootr00Gl  0.071971
4  arSLZs3c4o  0.058412

In [42]:
submit.to_csv('ensemble.csv', # file path, file name
              sep=',',   # seperator, delimiter (구분자),
              index = False,
              na_rep='NaN')

inference time : 약 3분